<a href="https://colab.research.google.com/github/goguma999/Portfolio/blob/main/hitterdetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
########################  라벨 바꾸기
import os

# 폴더 경로 지정
folders = [
    "/content/drive/MyDrive/batting/alltrain",
    "/content/drive/MyDrive/batting/allval"
]

# 각 폴더에서 khd로 시작하는 .txt 파일 내용을 수정
for folder_path in folders:
    count = 0  # 변경된 파일 수를 셀 변수
    for filename in os.listdir(folder_path):
        # 파일이 ooo로 시작하고 .txt 확장자인지 확인
        if filename.startswith("nsh") and filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)

            # 파일 읽기 및 첫 글자 수정
            with open(file_path, 'r+', encoding='utf-8') as file:
                content = file.read()
                if content:  # 내용이 있는 경우
                    # 첫 글자를 'n'으로 바꿔서 저장
                    new_content = '5' + content[1:]
                    file.seek(0)
                    file.write(new_content)
                    file.truncate()
                    #print(f"{file_path}: 첫 글자를 '5'로 변경했습니다.")
                    count += 1  # 변경된 파일 수 증가

    print(f"{folder_path} 폴더에서 총 {count}개의 파일 내용을 변경했습니다.\n")

print("모든 파일 내용 변경 완료!")


In [ ]:
import yaml

#데이터셋 경로 및 클래스 정보 설정
data_yaml = {
    'train': '/content/drive/MyDrive/batting/alltrain',  # Train 데이터 경로
    'val': '/content/drive/MyDrive/batting/allval',  # Validation 데이터 경로
    'nc': 6,  # 클래스 수
    'names': ['geonchang', 'yongkyu', 'heedong', 'daehyung', 'byungho', 'sihwan'] } # 클래스 이름

# yaml 파일 저장 경로 설정
yaml_file_path = '/content/drive/MyDrive/batting/data.yaml'

# data.yaml 파일 생성
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)

print(f"data.yaml 파일이 {yaml_file_path} 경로에 성공적으로 생성되었습니다.")

In [ ]:
# YOLO 라이브러리 설치
!pip install ultralytics

# YOLO 라이브러리 불러오기
from ultralytics import YOLO

# W&B 비활성화 (필수)
import os
os.environ['WANDB_MODE'] = 'offline'

# YOLOv8 모델 불러오기
model = YOLO('yolov8s.pt')

# 학습 데이터셋 경로 설정
data_yaml = '/content/drive/MyDrive/batting/data.yaml'

# 모델 학습 실행
model.train(
    data=data_yaml, #학습에 사용할 데이터 경로(data.yaml 파일)
    epochs=50, #학습을 반복할 횟수
    imgsz=480, #이미지 크기
    batch=8, #배치 크기
    name='hitter_detector', #프로젝트 이름
    workers=4)  # 데이터 로딩에 사용할 CPU 코어 수

model.save('/content/drive/MyDrive/batting/6_trained_model.pt')

In [ ]:
###################### 영상에 모델 적용하기 - 타겟을 감지하는지 확인
import cv2
import torch
from ultralytics import YOLO

# YOLO 모델을 로드합니다 (이미 로드된 상태라면 이 부분은 건너뛰세요)
model = YOLO('/content/drive/MyDrive/batting/6_trained_model.pt')


# 클래스 이름과 각각의 색상 설정 (BGR 형식)
class_names = ['geonchang', 'yonggyu', 'heedong', 'daehyung', 'byungho', 'sihwan']
colors = {
    "geonchang": (0, 255, 0),   # Green
    "yonggyu": (0, 0, 255),     # Red
    "heedong": (255, 0, 0),     # Blue
    "daehyung": (0, 255, 255), # Yellow
    "byungho": (128, 0, 128),  # Purple
    "sihwan": (255, 200, 100)     # Light Blue
}

# 동영상 파일 경로
video_path = '/content/drive/MyDrive/batting/testmp4/05gc.mp4'  # 동영상 경로를 적절히 수정하세요
output_video_path = '/content/drive/MyDrive/batting/mp4/all_output.mp4'  # 출력될 동영상 경로

# 동영상을 불러옵니다
cap = cv2.VideoCapture(video_path)

# 동영상의 프레임 크기 및 FPS를 가져옵니다
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 출력 동영상을 저장할 준비를 합니다
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 코덱 설정
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Confidence와 IOU 임계값을 설정
confidence_threshold = 0.65  # 원하는 신뢰도 임계값으로 조정
iou_threshold = 0.5         # 원하는 IOU 임계값으로 조정

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 동영상이 끝나면 루프를 종료합니다

    # YOLO 모델을 사용하여 현재 프레임에서 사물 검출을 수행합니다
    results = model(frame)

    # confidence_threshold 이상인 검출 결과만 그리도록 필터링
    for result in results:
        boxes = result.boxes
        for box in boxes:
            if box.conf >= confidence_threshold:
                # 박스 좌표 및 신뢰도 가져오기
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf.item()
                class_id = int(box.cls.item())

                # 클래스 인덱스가 유효한 경우에만 처리
                if class_id < len(class_names):
                    class_name = class_names[class_id]
                    label = f"{class_name} {confidence:.2f}"

                    # 클래스별 색상 선택 (기본은 파란색)
                    color = colors.get(class_name, (255, 0, 0))

                    # 박스를 그립니다
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    # 글씨를 크게 표시하고, 위치를 조정합니다
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    # 결과가 그려진 프레임을 출력 동영상에 저장합니다
    out.write(frame)


    # 결과를 실시간으로 확인하려면 아래 코드 사용 (원하는 경우)
    # cv2.imshow('Detected Frame', frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# 리소스 해제
cap.release()
out.release()
# cv2.destroyAllWindows()